In [1]:
import pandas as pd
from functions_load_and_transform import schedule, player_list, scoringDF

from datetime import datetime


In [2]:
def calc_score(row): # check if player guessed correctly
    if row[player] == row["Winner"]:
        return 1
    else:
        return 0

In [3]:
def check_winner(row): # winner column in game Data
    if row["Score Home"] > row["Score Guest"]:
        return row["Home Team"]
    else:
        return row["Away Team"]

In [12]:
schedule = pd.read_csv("data/schedule.csv", delimiter=";")
schedule["Date"] = pd.to_datetime(schedule["Date"], format="%Y.%m.%d %H:%M")
schedule["Score Home"] = 0
schedule["Score Guest"] = 0
schedule["Winner"] = schedule.apply(check_winner, axis=1)

def create_empty_bets_csv():
    betsDF = pd.DataFrame(columns=player_list)
    betsDF[["Week", "Game Nr."]] = schedule[["Week", "Game Nr."]]
    betsDF.to_csv("data/bets_2024.csv", index=False)

create_empty_bets_csv()
# betsDF = pd.concat([schedule, betsDF])

In [5]:
# scoringDF = pd.DataFrame(columns=player_list) # collect the points for each game
# scoringDF["Week"] = schedule["Week"]

# schedule


# for player in player_list:
#     scoringDF[player] = betsDF.apply(calc_score, axis=1)

In [6]:
with open("data/tipps.txt") as f:
    for line in f.readlines():
        x = line.split(";")
        print(x[1].split(","))

betsDF[player_list].head(20)

[" ['Kansas City Chiefs'", " 'Philadelphia Eagles'", " 'Atlanta Falcons'", " 'Buffalo Bills'", " 'Chicago Bears'", " 'Cincinnati Bengals'", " 'Indianapolis Colts'", " 'Miami Dolphins'", " 'New Orleans Saints'", " 'New York Giants'", " 'Las Vegas Raiders'", " 'Seattle Seahawks'", " 'Dallas Cowboys'", " 'Washington Commanders'", " 'Detroit Lions'", " 'San Francisco 49ers']\n"]
[" ['Kansas Chiefs'", " 'Eagles'", " 'Atlanta Falcons'", " 'Buffalo Bills'", " 'Chicago Bears'", " 'Cincinnati Bengals'", " 'Indianapolis Colts'", " 'Miami Dolphins'", " 'New Orleans Saints'", " 'New York Giants'", " 'Las Vegas Raiders'", " 'Seattle Seahawks'", " 'Dallas Cowboys'", " 'Washington Commanders'", " 'Detroit Lions'", " 'San Francisco 49ers']"]


,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def fill_in_bets(week_nr, player, bets):
    betsDF.loc[betsDF["Week"] == week_nr, player] = bets

def bets_input(week_nr, player, bets):
    df = pd.read_csv("data/bets_2024.csv")
    df.loc[df["Week"] == week_nr, player] = bets
    df.to_csv("data/bets_2024.csv", index=False)

In [8]:
week_nr = 1
player = "Ludwig"
testlist = ['Kansas City Chiefs', 'Philadelphia Eagles', 'Atlanta Falcons', 'Buffalo Bills', 'Chicago Bears', 'Cincinnati Bengals', 'Indianapolis Colts', 'Miami Dolphins', 'New Orleans Saints', 'New York Giants', 'Las Vegas Raiders', 'Seattle Seahawks', 'Dallas Cowboys', 'Washington Commanders', 'Detroit Lions', 'San Francisco 49ers']


bets_input(week_nr=week_nr, player=player, bets=testlist)

df = pd.read_csv("data/bets_2024.csv")

df.head(30)

,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang,Week,Game Nr.
0,NaN,NaN,NaN,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,NaN,NaN,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
2,NaN,NaN,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3
3,NaN,NaN,NaN,NaN,Buffalo Bills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4
4,NaN,NaN,NaN,NaN,Chicago Bears,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
5,NaN,NaN,NaN,NaN,Cincinnati Bengals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6
6,NaN,NaN,NaN,NaN,Indianapolis Colts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,7
7,NaN,NaN,NaN,NaN,Miami Dolphins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,8
8,NaN,NaN,NaN,NaN,New Orleans Saints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9
9,NaN,NaN,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10


In [10]:
bets_input(2, "Alex", testlist)
df = pd.read_csv("data/bets_2024.csv")

df.head(30)

,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang,Week,Game Nr.
0,NaN,NaN,NaN,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,NaN,NaN,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
2,NaN,NaN,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3
3,NaN,NaN,NaN,NaN,Buffalo Bills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4
4,NaN,NaN,NaN,NaN,Chicago Bears,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
5,NaN,NaN,NaN,NaN,Cincinnati Bengals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6
6,NaN,NaN,NaN,NaN,Indianapolis Colts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,7
7,NaN,NaN,NaN,NaN,Miami Dolphins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,8
8,NaN,NaN,NaN,NaN,New Orleans Saints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9
9,NaN,NaN,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10


In [11]:
bets_input(1, "Wolfgang", testlist)
df = pd.read_csv("data/bets_2024.csv")

df.head(30)

,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang,Week,Game Nr.
0,NaN,NaN,NaN,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,NaN,Kansas City Chiefs,1,1
1,NaN,NaN,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,Philadelphia Eagles,1,2
2,NaN,NaN,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,Atlanta Falcons,1,3
3,NaN,NaN,NaN,NaN,Buffalo Bills,NaN,NaN,NaN,NaN,NaN,NaN,Buffalo Bills,1,4
4,NaN,NaN,NaN,NaN,Chicago Bears,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Bears,1,5
5,NaN,NaN,NaN,NaN,Cincinnati Bengals,NaN,NaN,NaN,NaN,NaN,NaN,Cincinnati Bengals,1,6
6,NaN,NaN,NaN,NaN,Indianapolis Colts,NaN,NaN,NaN,NaN,NaN,NaN,Indianapolis Colts,1,7
7,NaN,NaN,NaN,NaN,Miami Dolphins,NaN,NaN,NaN,NaN,NaN,NaN,Miami Dolphins,1,8
8,NaN,NaN,NaN,NaN,New Orleans Saints,NaN,NaN,NaN,NaN,NaN,NaN,New Orleans Saints,1,9
9,NaN,NaN,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,New York Giants,1,10


In [9]:
# daten laden
# Tipps eintragen
# Scoring berechnen
# bets.csv überschreiben